In [1]:
import pandas as pd
import src.search_functions as sf
import src.cleaning as cl

In [2]:
# available conditions to search for:

num_emjpl = {"number_of_employees": {"$lt": 10000}}
founded_year = {"founded_year": {"$gt": 2007}}
money_raised_gt1M = {"total_money_raised": {"$regex": ".*[MB].*"}}
games_video = {"category_code": "games_video"}
web = {"category_code": "web"}
#web_and_games = {"category_code": ["games_video", "web"]}

In [3]:
q1_web_startups =sf.mongo_search(founded_year, money_raised_gt1M, web)
q1_web_startups.head(2)


,name,category_code,number_of_employees,founded_year,total_money_raised,city1,city2,lon1,lat1,lon2,lat2
0,Fixya,web,30.0,2013,$8M,San Mateo,None,-122.323895,NaN,-122.323895,NaN
42,Tongxue,web,NaN,2013,$6M,None,None,NaN,NaN,NaN,NaN


In [4]:
q1_games_startups =sf.mongo_search(founded_year, money_raised_gt1M, games_video)
q1_games_startups.head(2)

,name,category_code,number_of_employees,founded_year,total_money_raised,city1,city2,lon1,lat1,lon2,lat2
1,Social Gaming Network,games_video,100.0,2011,$17.1M,Los Angeles,Beverly Hills,-122.161523,34.069849,-122.161523,34.069849
6,Titan Gaming,games_video,18.0,2010,$1M,Santa Monica,None,-113.515769,NaN,-113.515769,NaN


In [5]:
startup_top10_cities = cl.merge_and_get_top10_cities(q1_web_startups,q1_games_startups)

In [6]:
startup_top10_cities

,City,Count
0,San Francisco,14
1,New York,12
2,Palo Alto,5
3,London,4
4,San Mateo,3
5,Los Angeles,3
6,Menlo Park,3
7,Tel Aviv,3
8,Istanbul,3
9,South Pasadena,2


In [7]:
q2_game =sf.mongo_search(games_video)
q2_web =sf.mongo_search(web)
top10_cities=cl.merge_and_get_top10_cities(q2_game,q2_web)

In [9]:
cities = cl.merge_cities(startup_top10_cities,top10_cities)
cities

,City,Count_startups,Count_comp
0,San Francisco,14,274
1,New York,12,258
2,Palo Alto,5,64
3,London,4,140
4,Los Angeles,3,90
